## Exemplifying notebook

In [22]:
import yaml
import itertools
import numpy as np
import pylab as plt
from math import *
from types import *
from operator import itemgetter
from scipy.special import hyp2f1
from inspect import getargspec
from iminuit import Minuit,util
%matplotlib inline

Below, the *ParseConfig* function returns a dictionary containing the name of the profile and its corresponding parameter values  
So config could contain an element named *'dm'* with subelement *'name':NFW* and *'r0':2*

In [ ]:
dm = ParseProfile(config,'dm')
st = ParseProfile(config,'st')
kr = ParseProfile(config,'kr')

In [44]:
cst = 8.*np.pi*4.3e-6
class SigmaLOS(object):
    def __init__(self, dm, st, kr):
        # with this syntax, the class is instantiated 
        # with a precise and fixed sequence of profiles
        self.dm = build_profile(**dm)
        self.st = build_profile(**st)
        self.kr = build_kernel(**kr)
    
    # alternatively (refers to the commented cell above)
    #def __init__(self, profiles):
    #    self.dm = build_profile(**profiles['dm'])
    #    self.st = build_profile(**profiles['st'])
    #    self.kr = build_kernel(**profiles['kr'])
    # this syntax allows more freedom in how the class is instantiated
        
    def integrand(self, x, gamma, alpha):
        return self.kr(x/gamma) * self.st.density(x) * self.dm.mass(x*alpha) / x
    
    def __call__(self, R, **free_params):
        self.dm.__dict__.update(**free_params)
        self.st.__dict__.update(**free_params)
        self.kr.__dict__.update(**free_params)
        gamma = R/self.st.rh
        alpha = self.st.rh/self.dm.r0
        integral = quad(self.integrand, gamma, np.inf, args=(gamma,alpha))
        return cst*integral[0]/self.st.surface_brightness(gamma)

The *integrand* and *call* method of the class above refer to the following calculation of $\sigma^2_{l.o.s.}$ (by Mamom and Lokas):
\begin{equation}
\sigma^2_{l.o.s.}(R) = \frac{2 G}{I(R)} \int^\infty_\gamma K\left(\frac{x}{\gamma},\frac{\delta}{\gamma},\beta\right) \nu(x) M(x \alpha) \frac{dx}{x}
\end{equation}
where
\begin{equation}
\alpha = \frac{r_h}{r_0} \quad , \quad \gamma = \frac{R}{r_h} \quad , \quad \delta = \frac{r_a}{r_h}
\end{equation}
These variable transformations ensure that the newly defined quantities, and thus the integral, are dimensionless.

In [ ]:
class loglike(object):    
    def __init__(self, data, sigma, like='Gauss'):
        self.R = data[0]
        self.v = data[1]
        self.dv = data[2]
        self.sigma = sigma
        self.like = like
    
    def GaussLike(self, **params):
        term1 = 0.5*np.power(self.v-self.v.mean(),2) / (self.sigma(self.R, **params))
        term2 = np.log(self.sigma(*params) + self.dv**2)
        return (term1 + term2).sum()
    
    def __call__(self, **freepars):
        if self.like=='Gauss':
            return self.GaussLike(**freepars)

When the *loglike* class is instantiated, an instance of the *SigmaLOS* is created. In turn, instances of the three terms entering the $\sigma^2_{l.o.s}$ are created. So when the flow should be:  
- an instance of *loglike* is called by the minimiser with a new set of free parameters, eg. `LL = loglike(data, SigmaLOS(dm,st,kr))`  
- this calls the loglikelihood function (defaulted to *Gaussian* here), eg. `LL(freeparams)`  
- the instance of the *SigmaLOS* is called with the new parameters  
- the internal dictionaries of *dark matter*, *stellar* and *kernel* functions instances are updated  
- the integral is calculated for the new parameters and the results returned